# Video Presentation

If you uploaded your Video Presentation to Bluejeans, YouTube, or any other streaming services, please provide the link here:


*   [Video Presentation Link](https://www.youtube.com/watch?v=iXs0hT1i-1A)


Make sure the video sharing permissions are accessible for anyone with the provided link.

In [3]:
def data_parser():
    import pandas as pd
    import numpy as np

    df = pd.read_csv('Contracts.csv', header=0, low_memory=False)  # Added low_memory=False to address the DtypeWarning

    column_list = [
        'federal_action_obligation', 'total_dollars_obligated', 'current_total_value_of_award',
        'action_date', 'action_date_fiscal_year', 'period_of_performance_start_date',
        'period_of_performance_current_end_date', 'awarding_agency_name',
        'awarding_sub_agency_name', 'funding_sub_agency_name', 'recipient_name',
        'recipient_city_name', 'recipient_county_name', 'recipient_state_name',
        'product_or_service_code_description', 'naics_description'
    ]
    df = df[column_list]

    df["recipient_name"] = np.where(
        df["recipient_name"].str.contains("Lockheed", case=False, na=False),
        "Lockheed Martin",
        df["recipient_name"]
    )

    # Changed to handle mixed date formats
    df["action_date"] = pd.to_datetime(df["action_date"], format='mixed', errors='coerce')
    df = df.sort_values(by="action_date").reset_index(drop=True)

    df["action_date"] = df["action_date"].dt.strftime("%m/%d/%Y")
    
    df['Daily Total Obligation'] = df.groupby('action_date')['federal_action_obligation'].transform('sum')

    df.to_csv('Filtered_Contracts.csv', index=False)

    return df

data_parser()

,federal_action_obligation,total_dollars_obligated,current_total_value_of_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_name,awarding_sub_agency_name,funding_sub_agency_name,recipient_name,recipient_city_name,recipient_county_name,recipient_state_name,product_or_service_code_description,naics_description,Daily Total Obligation
0,31934161.00,7.823005e+07,1.167745e+08,10/01/2022,2023,10/1/2022,2/15/2025,Department of Defense,Department of the Air Force,Department of the Air Force,Lockheed Martin,FORT WORTH,TARRANT,TEXAS,MAINT/REPAIR/REBUILD OF EQUIPMENT- AIRCRAFT CO...,AIRCRAFT MANUFACTURING,31934161.00
1,0.00,2.000247e+07,1.992292e+07,10/02/2022,2023,1/31/2025,1/31/2025,Department of Defense,Department of the Air Force,Department of the Air Force,Lockheed Martin,ORLANDO,ORANGE,FLORIDA,"GUIDED MISSILE SYSTEMS, COMPLETE",GUIDED MISSILE AND SPACE VEHICLE MANUFACTURING,0.00
2,0.00,4.686414e+08,4.888159e+08,10/03/2022,2023,7/29/2011,12/14/2022,Department of Defense,Missile Defense Agency,Missile Defense Agency,Lockheed Martin,LITTLETON,JEFFERSON,COLORADO,GUIDED MISSILE AND SPACE VEHICLE EXPLOSIVE PRO...,"RESEARCH AND DEVELOPMENT IN THE PHYSICAL, ENGI...",58692310.13
3,2500000.00,6.042954e+08,7.318882e+08,10/03/2022,2023,4/29/2021,9/30/2025,Department of Defense,Department of the Army,Department of the Army,Lockheed Martin,GRAND PRAIRIE,DALLAS,TEXAS,NATIONAL DEFENSE R&D SERVICES; DEPARTMENT OF D...,GUIDED MISSILE AND SPACE VEHICLE MANUFACTURING,58692310.13
4,0.00,2.010270e+04,2.010270e+04,10/03/2022,2023,1/14/2022,1/16/2023,Department of Defense,Defense Logistics Agency,Defense Logistics Agency,"DERCO AEROSPACE, INC.",MILWAUKEE,MILWAUKEE,WISCONSIN,AIRFRAME STRUCTURAL COMPONENTS,OTHER AIRCRAFT PARTS AND AUXILIARY EQUIPMENT M...,58692310.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114440,342.37,3.423700e+02,3.423700e+02,07/04/2025,2025,7/4/2025,7/7/2025,Department of Defense,Defense Logistics Agency,Defense Logistics Agency,SIKORSKY AIRCRAFT CORPORATION,STRATFORD,GREATER BRIDGEPORT,CONNECTICUT,AIRFRAME STRUCTURAL COMPONENTS,AIRCRAFT ENGINE AND ENGINE PARTS MANUFACTURING,30116.79
114441,1196.16,1.196160e+03,1.196160e+03,07/04/2025,2025,7/4/2025,12/15/2026,Department of Defense,Defense Logistics Agency,Defense Logistics Agency,Lockheed Martin,ORLANDO,ORANGE,FLORIDA,"BUSHINGS, RINGS, SHIMS, AND SPACERS","BOLT, NUT, SCREW, RIVET, AND WASHER MANUFACTURING",30116.79
114442,21255.22,2.125522e+04,2.125522e+04,07/04/2025,2025,7/4/2025,2/11/2026,Department of Defense,Defense Logistics Agency,Defense Logistics Agency,SIKORSKY AIRCRAFT CORPORATION,STRATFORD,GREATER BRIDGEPORT,CONNECTICUT,MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,INDUSTRIAL VALVE MANUFACTURING,30116.79
114443,743.68,7.436800e+02,7.436800e+02,07/04/2025,2025,7/4/2025,7/7/2025,Department of Defense,Defense Logistics Agency,Defense Logistics Agency,SIKORSKY AIRCRAFT CORPORATION,STRATFORD,GREATER BRIDGEPORT,CONNECTICUT,"BEARINGS, PLAIN, UNMOUNTED",BALL AND ROLLER BEARING MANUFACTURING,30116.79


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np

def web_parser1():
    og_url = "https://stooq.com/q/d/?s=lmt.us&i=d&f=20220101&t=20251004&l={page}"
    data = []

    for page in range(1, 25):
        response = requests.get(og_url.format(page=page))
        soup = BeautifulSoup(response.text, 'html.parser')

        for row in soup.find_all("tr"):
            cols = row.find_all('td')
            if len(cols) == 9:
                date = cols[1].text.strip()

                try:
                    datetime.strptime(date, "%d %b %Y")
                except ValueError:
                    continue

                open = float(cols[2].text.strip().replace(',', ''))
                high = float(cols[3].text.strip().replace(',', '')) 
                low = float(cols[4].text.strip().replace(',', '')) 
                close = float(cols[5].text.strip().replace(',', '')) 
                percent = float(cols[6].text.strip().replace('%', '').replace(',', '')) if cols[7].text.strip() else 0.0
                change = float(cols[7].text.strip().replace(',', '')) if cols[7].text.strip() else 0.0 #the earliest row has no values
                volume = float(cols[8].text.strip().replace(',', '')) 
                
                data.append([date, open, high, low, close, percent, change, volume])

    df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close", "Percent Change", "Change", "Volume"])

    df["Date"] = pd.to_datetime(df["Date"], format="%d %b %Y").dt.strftime('%m/%d/%Y')

    df.to_csv('stockprice.csv', index=False)
    
    return df


web_parser1()

,Date,Open,High,Low,Close,Percent Change,Change,Volume
0,10/03/2025,499.370,506.810,499.245,504.490,1.03,5.130,1006534.0
1,10/02/2025,499.000,503.600,497.010,499.360,0.07,0.360,915832.0
2,10/01/2025,496.470,504.070,495.000,499.000,-0.04,-0.210,1187448.0
3,09/30/2025,492.580,499.890,491.140,499.210,1.47,7.230,1312722.0
4,09/29/2025,497.000,498.986,489.220,491.980,0.93,4.540,1391134.0
...,...,...,...,...,...,...,...,...
937,01/07/2022,349.888,353.704,348.986,350.928,0.60,2.090,1677570.0
938,01/06/2022,350.791,351.997,348.406,348.838,-0.04,-0.138,1404944.0
939,01/05/2022,353.714,354.685,348.711,348.976,-1.06,-3.748,1726386.0
940,01/04/2022,346.435,354.234,346.121,352.724,2.15,7.428,1396860.0


In [5]:
import requests
import pandas as pd

def web_parser2():
  response = requests.get('https://www.alphavantage.co/query?function=ALUMINUM&interval=monthly&apikey={API_KEY}') 
  data = response.json() 


  df = pd.DataFrame(data['data']) 
  df.columns = ['Date', 'Price/Ton'] 


  df = df[df['Price/Ton'] != '.'] 
  df['Date'] = pd.to_datetime(df['Date'])
  df["Date"] = pd.to_datetime(df["Date"], format="%Y %m %d").dt.strftime('%m/%d/%Y')
  
  df['Price/Ton'] = df['Price/Ton'].astype(float)
  df['Change'] = df['Price/Ton'].diff(-1).fillna(0)
  df['Percent Change'] = df['Price/Ton'].pct_change(-1).fillna(0) * 100
  
  df.to_csv('aluminum_prices.csv', index=False) 
  return df
web_parser2()

,Date,Price/Ton,Change,Percent Change
0,06/01/2025,2525.959048,78.614957,3.212256
1,05/01/2025,2447.344091,79.543636,3.359389
2,04/01/2025,2367.800455,-283.798117,-10.702907
3,03/01/2025,2651.598571,-4.173929,-0.157164
4,02/01/2025,2655.772500,84.402065,3.282377
...,...,...,...,...
421,05/01/1990,1527.000000,1.000000,0.065531
422,04/01/1990,1526.000000,-41.000000,-2.616465
423,03/01/1990,1567.000000,113.000000,7.771664
424,02/01/1990,1454.000000,-74.000000,-4.842932


# Topic Summary

Please provide a brief executive summary (5 sentences or less) discussing your topic:

Our analysis concentrates on Lockheed's government contracts granted by the Department of Defense; how do these contracts impact Lockheed's stock prices? We also consider the influence of global aluminum prices on Lockheed's stock prices. Our goal is to synthesize information from the awarded contracts data, along with the commodity price trend, to offer explanations for changes in Lockheed's stock prices. In particular, we want to know if large government contracts affect an increase in stock prices to a larger extent. This provides insight into the impact of exogenous events and materials costs and the way in which they contribute to investor perceptions and subsequent valuations. Overall we wanted to look at multiple factors including contracts, commodities, geographic footprint, and contract timing to draw insights on what investors could potenially use when investing in Lockheed.

## Insights

In [6]:
from sklearn.tree import DecisionTreeRegressor

def insight1():
    contracts = pd.read_csv('Filtered_Contracts.csv')  
    

    contracts['action_date'] = pd.to_datetime(contracts['action_date'])
    
    daily = contracts.groupby('action_date').size().reset_index(name='Count')
    

    daily['Month'] = daily['action_date'].dt.month
   

    X = daily[['Month']].values
    

    y = daily['Count'].values
  

    tree = DecisionTreeRegressor(max_depth=5, random_state=42)
    

    tree.fit(X, y)
    

    print("PREDICTED DAILY CONTRACTS BY MONTH")
    for month in range(1, 13):
        prediction = tree.predict([[month]])[0]
        

        month_name = pd.Timestamp(month=month, year=1, day=1).month_name()
        

        print(f"{month_name}: {int(prediction)} contracts")
        

insight1()


PREDICTED DAILY CONTRACTS BY MONTH
January: 106 contracts
February: 120 contracts
March: 120 contracts
April: 120 contracts
May: 113 contracts
June: 110 contracts
July: 102 contracts
August: 133 contracts
September: 141 contracts
October: 101 contracts
November: 106 contracts
December: 108 contracts


### Insight 1 Explanation

Insight:
The insight drawn is a prediction of daily contract activity patterns based on the month of the year. Using a Decision Tree Regressor, we identified seasonal trends in when Lockheed Martin receives federal contracts. This is meaningful because it reveals procurement cycles and budget periods in government contracting. The highest prediction was for September, coinciding with the fiscal year-end, suggesting the government increases spending to use up their annual budget before it expires.

Relation to Topic:
Our topic was analyzing Lockheed Martin's federal contracts and stock performance. This analysis shows contract distribution patterns and how government procurement timing might influence business operations and stock performance during peak contract months. This allows investors to know what months to pay attention to Lockheed the most.

New Learning:
To perform this analysis we had to learn how to use Decision Tree Regressors for time-based predictions, extract temporal features (like month) from datetime objects, and aggregate data by date to count daily occurrences.

In [7]:
def insight2():
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split

    df = pd.read_csv("stockprice.csv")
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    df["Previous_Close"] = df["Close"].shift(1)
    df["MA_60"] = df["Close"].rolling(60).mean()
    df["MA_180"] = df["Close"].rolling(180).mean()
    df["Year"] = df["Date"].dt.year

    df = df.dropna().reset_index(drop=True)

    features = ["Year", "Previous_Close", "MA_60", "MA_180"]
    X = df[features]
    y = df["Close"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, shuffle=False
    )

    model = LinearRegression()
    model.fit(X_train, y_train)

    r2 = model.score(X_test, y_test)
    print(f"R² Score: {r2:.4f}")


    last_date_str = df["Date"].iloc[-1].strftime("%Y-%m-%d")

 
    year, month, day = last_date_str.split("-")
    day = int(day)  
    predictions = []

    prev_close = df["Close"].iloc[-1]
    ma60 = df["MA_60"].iloc[-1]
    ma180 = df["MA_180"].iloc[-1]
    yr = df["Year"].iloc[-1]

    for i in range(1, 6):
        new_day = day + i 

        next_date = f"{year}-{month}-{new_day}"

        X_future = pd.DataFrame([[yr, prev_close, ma60, ma180]], columns=features)
        predicted_price = model.predict(X_future)[0]

        predictions.append((next_date, predicted_price))

        prev_close = predicted_price
        ma60 = (ma60 * 59 + predicted_price) / 60
        ma180 = (ma180 * 179 + predicted_price) / 180

    print("\nNext 5 Days Predictions:")
    for date, price in predictions:
        print(f"{date} → ${price:.2f}")


insight2()


R² Score: 0.8836

Next 5 Days Predictions:
2025-10-4 → $505.25
2025-10-5 → $506.00
2025-10-6 → $506.73
2025-10-7 → $507.45
2025-10-8 → $508.15


### Insight 2 Explanation

Insight: We used a Linear Regression model with features such as the previous close and 60-/180-day moving averages to develop a very short-term forecast for Lockheed Martin's stock price. The model updates the trend indicators predicted for the next five days, showing how the most recent momentum affected the price direction. This showed that Lockheed was on an upward trend at the time of data collection

Relation to Topic: This forecast helps establish a baseline of "normal" stock behavior based solely on price trends, absent any effects related to contract events. The baseline in a real scenario could be used to compare against stock movement after an event. Also since we were looking at Lockheed's stock price this would be the most relevant prediction to make.

New Learning: We learned how to engineer time-series features, fit Linear Regression to stock data, and then forward-predict iteratively by dynamically updating the moving averages for each future day. 


In [8]:
import pandas as pd

def insight3():
    """Calculate correlation between aluminum volatility and Lockheed stock price"""
    
    
    aluminum_df = pd.read_csv("aluminum_prices.csv")
    stock_df = pd.read_csv("stockprice.csv")
    
    
    aluminum_df['Month'] = pd.to_datetime(aluminum_df['Date']).dt.to_period('M')
    stock_df['Month'] = pd.to_datetime(stock_df['Date']).dt.to_period('M')
    
    
    monthly_aluminum = aluminum_df.groupby('Month')['Price/Ton'].mean().reset_index()
    monthly_stock = stock_df.groupby('Month')['Close'].mean().reset_index()
    
 
    merged = pd.merge(monthly_aluminum, monthly_stock, on='Month', how='inner')
    
    
    merged['Aluminum_pct_change'] = merged['Price/Ton'].pct_change() * 100
    merged['Stock_pct_change'] = merged['Close'].pct_change() * 100
    
 
    merged_clean = merged.dropna()
    
    
    corr = merged_clean['Aluminum_pct_change'].corr(merged_clean['Stock_pct_change'])
    
   
    strength = "strong" if abs(corr) > 0.5 else "weak"
    
    print(f"Correlation: {corr:.3f} ({strength})")


insight3()

Correlation: 0.337 (weak)


### Insight 3 Explanation


Insight: We wanted to look at how volatility in aluminum affected the volatility of the stock. Since we were able to quantify the monthly percentage change in both price metrics, we could measure whether swings in a raw material cost parallels with stock performance. This is meaningful, because aluminum is a key input used in aerospace manufacturing, which has a direct impact the production costs, and ultimately profit margin. For example the correlation coefficient indicates whether the two metrics move together (positive), move opposite (negatively), or move independently (near zero). We found that the coefficient was positive and weak making it not something to invest based on.

Relation to Topic: Our topic was performing an analysis on Lockheed Martin's stock performance compared to comodity cost. This analysis intersects raw material approach to economics, and then towards stock performance, and additionally has the potential to show how external market factors impact financial health and confidence to the investor.

New Learning: In order to conduct the analysis, we learned to align time-series data from various economic data sources to the same time period (monthly) and comment on percentage changes to measure volatility across different scales (comparing aluminum and Lockheed stocks). Moreover, we learned to use the Pearson correlation coefficient to quantify the relation of two variables and the strength of that relation. 

In [9]:
import pandas as pd

def insight4():
    df = pd.read_csv('Filtered_Contracts.csv')
    df = df.dropna(subset=['federal_action_obligation', 'recipient_state_name', 'recipient_name'])
    
    
    top_5_states = df.groupby('recipient_state_name')['federal_action_obligation'].sum().nlargest(19).index
    
    
    df_top5 = df[df['recipient_state_name'].isin(top_5_states)]
    
    
    contract_value_df = df_top5.pivot_table(
        index='recipient_state_name',
        columns='recipient_name',
        values='federal_action_obligation',
        aggfunc='sum',
        fill_value=0
    )
    
    
    contract_value_df['Total'] = contract_value_df.sum(axis=1)
    contract_value_df = contract_value_df.sort_values('Total', ascending=False)
    contract_value_df = contract_value_df.drop('Total', axis=1)
    
    print(contract_value_df)
    return contract_value_df

insight4()

recipient_name        ASTROTECH SPACE OPERATIONS, LLC  DERCO AEROSPACE, INC.  \
recipient_state_name                                                           
TEXAS                                             0.0                   0.00   
FLORIDA                                    79248829.0                   0.00   
CONNECTICUT                                       0.0                   0.00   
NEW YORK                                          0.0                   0.00   
COLORADO                                          0.0                   0.00   
NEW JERSEY                                        0.0                   0.00   
GEORGIA                                           0.0                   0.00   
CALIFORNIA                                        0.0                   0.00   
ALABAMA                                           0.0                   0.00   
ARIZONA                                           0.0                   0.00   
KENTUCKY                                

recipient_name,"ASTROTECH SPACE OPERATIONS, LLC","DERCO AEROSPACE, INC.",ENFLITE LLC,LIFEPORT LLC,LONGBOW LLC,Lockheed Martin,OAO CORPORATION,SIKORSKY AIRCRAFT CORPORATION,SIKORSKY SUPPORT SERVICES INC,"SIKORSKY SUPPORT SERVICES, INC."
recipient_state_name,,,,,,,,,,
TEXAS,0.0,0.00,389950.0,0.0,0.000000e+00,8.257650e+10,0.0,0.000000e+00,0.00,0.00
FLORIDA,79248829.0,0.00,0.0,0.0,1.248392e+08,1.787510e+10,0.0,0.000000e+00,0.00,-280.98
CONNECTICUT,0.0,0.00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,1.278655e+10,-1070895.25,0.00
NEW YORK,0.0,0.00,0.0,0.0,0.000000e+00,9.141983e+09,0.0,0.000000e+00,0.00,0.00
COLORADO,0.0,0.00,0.0,0.0,0.000000e+00,7.415479e+09,0.0,0.000000e+00,0.00,0.00
NEW JERSEY,0.0,0.00,0.0,0.0,0.000000e+00,6.254243e+09,0.0,0.000000e+00,0.00,0.00
GEORGIA,0.0,0.00,0.0,0.0,0.000000e+00,5.151818e+09,0.0,0.000000e+00,0.00,0.00
CALIFORNIA,0.0,0.00,0.0,0.0,0.000000e+00,5.011390e+09,0.0,0.000000e+00,0.00,0.00
ALABAMA,0.0,0.00,0.0,0.0,0.000000e+00,3.753626e+09,0.0,0.000000e+00,0.00,0.00


### Insight 4 Explanation

Insight:
The insight we gained was a geographic breakdown of the distribution of Lockheed Martin's federal contracts over the top 19 states, by individual Lockheed's subsidiaries. We created a pivot table mapping each contract value against a lockheed entity within each state, which allowed us to see where defense spending is clustered as well as which divisions of the company operate out of each state. This is important, as it highlights their major operational locations as well as the economic implications of defense contracts on each state in the US. This means that states with higher contract values are also likely the location of larger production facilities or R&D functions, indicating Lockheed's strategic operational footprint. Our insight shows that Texas is Lockheed's biggest state.

Relation to Topic:
Our topic was analyzing Lockheed Martin's federal contracts and share price performance. The analysis we performed is a direct examination of the geographic distribution of contracts, across the country, so we could see where the federal government is exhibiting spending in defense contracts. For example, if some big policy change happened in Texas for the aerospace industry, investors should most likely pay attention to their Lockheed investments because majority of their footprint is in Texas. 

New Learning:
To perform the analysis we needed to learn how to perform pivot tables in pandas to change the way the data was organized from long-format to wide-format. We also learned how to do multi-dimensional grouping in one command to aggregate a pivot table by multiple categories (state and company) at one time. Finally, we learned how to sort a dataframe by first calculating temporary column totals (specifically rows) while maintaining detailed breakdown of the data.

In [10]:
import pandas as pd

def insight5():
    contracts = pd.read_csv('Filtered_Contracts.csv')
    stock = pd.read_csv('stockprice.csv')
    
    contracts['action_date'] = pd.to_datetime(contracts['action_date'])
    stock['Date'] = pd.to_datetime(stock['Date'])
    stock = stock.sort_values('Date')
    
    large = contracts[contracts['federal_action_obligation'] >= 500e6]
    
    results = []
    for _, row in large.iterrows():
        contract_date = row['action_date']
        future_stock = stock[stock['Date'] >= contract_date].head(6)
        if len(future_stock) >= 6:
            start_price = future_stock.iloc[0]['Close']
            end_price = future_stock.iloc[5]['Close']
            return_5d = ((end_price - start_price) / start_price) * 100
            results.append(return_5d)
    
    
    average_yield = sum(results) / len(results)
    positive_count = sum(1 for r in results if r > 0)
    negative_count = len(results) - positive_count
    
    print(f"Average 5-day yield: {average_yield:.2f}%")
    print(f"Positive returns: {positive_count}")
    print(f"Negative returns: {negative_count}")

insight5()

Average 5-day yield: -0.88%
Positive returns: 17
Negative returns: 25


### Insight 5 Explanation

Insight: 
This insight is a short-term investigation of stock returns after large federal contracts are awarded. By analyzing contracts above $500 million and examining stock returns over the 5 days after each award, we were able to observe how the federal government affects short-term stock market behavior. This is important because it measures a real market reaction, and in this case, provided evidence that, on average, recently awarded contracts produced slightly negative returns.

Relation to Topic:
Our topic was analyzing Lockheed's stock in the context of government contracts, and this insight presents an event-driven financial knowledge insight that links stock performance in the short term (in our case, 5 days) with the public sector (the awarded federal contract). 

New Learning: 
In order to be able to conduct this analysis, we had to learn how to date the contract align dates with stock market dates and then filter for large contracts. We also had to iteratively calculate short-term returns over multiple events which taught us how to abstract meaningful patterns from event driven data.

## Data Visualizations

In [11]:
import pandas as pd
import plotly.express as px

def visual_1():
    aluminum_df = pd.read_csv("aluminum_prices.csv")
    stock_df = pd.read_csv("stockprice.csv")
    
    aluminum_df['Date'] = pd.to_datetime(aluminum_df['Date'])
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    
    stock_df['Month'] = stock_df['Date'].dt.to_period('M')
    monthly_stock = stock_df.groupby('Month')['Close'].mean().reset_index()
    
    aluminum_df['Month'] = aluminum_df['Date'].dt.to_period('M')
    monthly_aluminum = aluminum_df.groupby('Month')['Price/Ton'].mean().reset_index()
    
    merged = pd.merge(monthly_aluminum, monthly_stock, on='Month', how='inner')
    
    merged['Aluminum_pct_change'] = merged['Price/Ton'].pct_change() * 100
    merged['Stock_pct_change'] = merged['Close'].pct_change() * 100
    merged = merged.dropna()
    
    merged['Month'] = merged['Month'].astype(str)
    
    fig = px.line(
        merged,
        x='Month',
        y=['Aluminum_pct_change', 'Stock_pct_change'],
        labels={
            'value': 'Monthly % Change',
            'Month': 'Month'
        },
        title='Monthly % Change: Aluminum vs Lockheed Stock',
        template='plotly_white'
    )
    
    fig.update_layout(height=600)
    fig.show()
    
    corr = merged['Aluminum_pct_change'].corr(merged['Stock_pct_change'])
    print(f"Correlation: {corr:.3f}")

visual_1()


Correlation: 0.337


### Visualization 1 Explanation

Insight:
The insight gained is the connection between changes in aluminum prices and changes in Lockheed's stock prices. By measuring the monthly percentage change in aluminum prices relative to Lockheed's stock, it is possible to observe whether and how changes in the price of a key input material affect stock performance. This is meaningful because aluminum is a critical input in aerospace manufacturing, and suppliers may increase their prices based on their own increased costs. Since aluminum is a key input, increases costs may reduce production margins, and ultimately investor sentiment. However, we found there is weak correlation.

Relation to Topic:
Our topic was simply the measurement of Lockheed's stock, and this analysis shows a relationship between material costs and stock performance. This illustrates how external economic realities, such as commodity prices, could have an affect on the valuation of the company. Lastly, the analysis shows a slight positive correlation showing that investors should pay some attention to input costs for Lockheed.

New Learning:
To conduct this analysis, we learned how to calculate monthly percentage changes, filter and merge datasets into monthly periods, and create scatter plots with trend lines in Plotly. We also learned how to quantify the relationship between two different variables using a measure of correlation, which is important when analyzing dependencies in financial data. We also learned how to format the graph properly.

In [12]:
import pandas as pd
import plotly.express as px

def visual_2():
    df = pd.read_csv('Filtered_Contracts.csv')
    df = df.dropna(subset=['federal_action_obligation', 'recipient_state_name', 'recipient_name'])
    
    df['federal_action_obligation_B'] = df['federal_action_obligation'] / 1e9
    
    top_14_states = df.groupby('recipient_state_name')['federal_action_obligation_B'].sum().nlargest(14).index #We chose 14 because only 14 states had significant contract values that showed up visually
    
    df_top14 = df[df['recipient_state_name'].isin(top_14_states)]  
    

    state_company_totals = df_top14.groupby(['recipient_state_name', 'recipient_name'])['federal_action_obligation_B'].sum().reset_index()
   
    state_order = state_company_totals.groupby('recipient_state_name')['federal_action_obligation_B'].sum().sort_values(ascending=False).index
    
    state_company_totals = state_company_totals[state_company_totals['federal_action_obligation_B'] > 0]
    
    fig = px.bar(state_company_totals, 
                 y='recipient_state_name', 
                 x='federal_action_obligation_B',
                 color='recipient_name',
                 labels={'federal_action_obligation_B': 'Contract Value ($ in Billions)', 
                         'recipient_state_name': 'State',
                         'recipient_name': 'Company'},
                 title='Contract Distribution Across Top 14 States by Company',
                 category_orders={'recipient_state_name': list(state_order)})
    
    fig.update_layout(showlegend=True, height=800)
    fig.show()

visual_2()

### Visualization 2 Explanation

Insight: 
This visualization depicts the distribution of federal contracts awarded to Lockheed Martin subsidiaries by state. Aggregate contract amounts will be shown by state and recipient company (Lockheed subsidiaries) to show both where federal dollars are concentrated and which Lockheed divisions have a presence in each state. This is important because state policy, such as tax incentives or regulations, can determine where major contractors choose to operate. We decided to present only 14 states, as the remaining state contract values were insignificant and not readily seen in visualization.


Relation to Topic: 
We were analyzing Lockheed's data both in terms of their federal contracts and the performance of their stock value. The visualization illustrates the geographic distribution of their contract awards, and illustrates the differentiated operations of Lockheed subsidiaries are strategically targeting states and state policy factors as they relate to allocating federal contract dollars. This can help investors know what state policies to pay attention to and what subsidaries have a bigger impact.


New Learning: 
We learned how to create stacked bar charts in Plotly that display multiple dimensions simultaneously (state, company, and contract value). We learned how to filter, aggregate, and group our data by multiple grouping variable, and order categorical data by computed totals towards a more intuitive visualization.

In [13]:
import pandas as pd
import plotly.express as px

def visual_3():
    contracts = pd.read_csv('Filtered_Contracts.csv')
    stock = pd.read_csv('stockprice.csv')
    
    contracts['action_date'] = pd.to_datetime(contracts['action_date'])
    stock['Date'] = pd.to_datetime(stock['Date'])
    stock = stock.sort_values('Date')
    
    large = contracts[contracts['federal_action_obligation'] >= 500e6]
    
    results = []
    for _, row in large.iterrows():
        contract_date = row['action_date']
        
        future_stock = stock[stock['Date'] >= contract_date].head(6)  
        if len(future_stock) >= 6:
            start_price = future_stock.iloc[0]['Close']
            end_price = future_stock.iloc[5]['Close']
            return_5d = ((end_price - start_price) / start_price) * 100
            results.append({'Amount ($ in Billions)': row['federal_action_obligation'] / 1e9, 'Return (%)': return_5d})
    
    df = pd.DataFrame(results)
    
    if len(df) == 0:
        return
        
    df['Color'] = ['Positive' if x > 0 else 'Negative' for x in df['Return (%)']]
    fig = px.scatter(df, x='Amount ($ in Billions)', y='Return (%)', color='Color',
                     color_discrete_map={'Positive': 'green', 'Negative': 'red'},
                     title='Contracts >$500M: 5-Day Stock Returns')
    fig.add_hline(y=0)
    fig.show()

visual_3()

### Visualization 3 Explanation

Insight:
The insight is a visual showing the effect of large federal contracts (greater than $500 million) on short-term stock returns. By plotting the 5-day return to contract size, and distinguishing between contracts that were associated with positive and negative outcomes, we are able to observe market patterns in response to federal spending on particular contracts.  This is significant since many investors are focused on identifying patterns linking government contracts and now know that large contracts actually cause more negative returns than positive.

Relation to Topic:
In our topic, we were interested in Lockheed’s stock and federal contracts, and the visualization demonstrates event-driven stock return performance. 

New Learning:
In analyzing the data, we learned to code the color of data points based on whether the return was positive or negative and to show event-driven data in a plot using Plotly. We also learned how to align contract dates with stock market dates and a visual, and summarized it in the project.